In [2]:
#import the necessary libraries
import numpy as np
from matplotlib import pyplot as pl
from scipy.integrate import quad #accurate integration function

# Problem 1

In [46]:
truth = 2 * (1 - 5**-0.5)

In [41]:
# Function for rectangle rule for integration
# Inputs: - f: 1d function to integrate
#         - a: lower bound
#         - b: upper bound
#         - n: number of intervals to divide the integration into
# Returns: - tot: the definite integral result
#          - err: an estimate of the error on tot
#          - dx: the interval width
def rect(f, a, b, n):
    #initialize the integral and error
    tot, err = 0, 0

    #compute the interval width
    dx = (b - a)/n

    #initialize the independent variable to the lower bound
    x = a

    #initalize a value to store an additional function call
    #to avoid doubled function calls when estimating error
    next_val = f(a)

    #initialize variable for function values
    val = 0

    #loop to upper bound
    while x < b:
        #shift the next function value to be current
        val = next_val
        #add function value to running sum (without dx factor)
        tot += val
        #increment independent variable
        x += dx
        #compute the next function value
        next_val = f(x)
        #estimate the error using approximate area of triangle
        #formed by the current and next function values
        err += dx * abs(val-next_val) / 2

    #returning while multiplying by common dx factor
    return tot*dx, err, dx

# Function for the rectangle rule implemented using numpy vectorization
# (Performs the same computation as rect)
# Inputs: - f: 1d function to integrate
#         - a: lower bound
#         - b: upper bound
#         - n: number of intervals to divide the integration into
# Returns: - the definite integral result
#          - err: an estimate of the error on tot
def rect_vec(f, a, b, n):
    #initialize independent variable array n long [a,b-dx]
    #holding the left edges of the rectangles
    xs = np.linspace(a, b, n+1)[:-1]
    #compute rectangle widths
    dx = (xs[1]-xs[0])

    #evaluate the function at all the x values "simultaneously" using 
    #  numpy vectorization
    #the vectorize function works but I found it increases runtime
    #  substantially due to apparent overhead
    #for simple arithmetic functions, numpy's built in vectorization
    #  does not incur this additional overhead
    ## vals = np.vectorize(f)(xs)
    vals = f(xs)
    
    #estimate error using the same idea as rect() but using array vectorization
    err = 0.5 * np.sum(np.abs(vals[1:]-vals[:-1])) * dx
    
    return np.sum(vals)*dx, err


# Function 
# Inputs: - f: 1d function to integrate
#         - a: lower bound
#         - b: upper bound
#         - n: number of intervals to divide the integration into
# Returns: - tot: the definite integral result
#          - err: an estimate of the error on tot
def rect_cen(f, a, b, n):
    #initialize the integral and error
    tot, err = 0, 0
    #compute the interval width
    dx = (b - a)/n
    #initializing x to be the center of the rectangle
    #  rather than the left edge
    x = a + dx/2
    #initialize variables to store current and upcoming function
    #  values to avoid duplicated function calls
    val = 0
    next_val = f(x)

    #estimate error using the areas of triangles formed at the 
    #  beginning and end of the integration interval
    err = abs(next_val - f(a)) / 2 * dx

    #loop over the interval of integration
    while x < b:
        #using the function value computed previously
        val = next_val
        #adding to the running sum
        tot += val
        #incrementing x
        x += dx
        #computing next function value
        next_val = f(x)

    #returning while multiplying by common dx factor
    return tot*dx, err

# Function for the trapezoid rule for integration
# Inputs: - f: 1d function to integrate
#         - a: lower bound
#         - b: upper bound
#         - n: number of intervals to divide the integration into
# Returns: - tot: the definite integral result
#          - err: an estimate of the error on tot
def trap(f, a, b, n):
    #initialize the integral
    tot = 0
    #compute the interval width
    dx = (b - a)/n
    #initializing x to its second value since next_val will
    #  already hold f(a+dx) so we don't want to duplicate this
    #  after the first loop evaluation
    x = a+dx
    #initialize variables to store current and upcoming function
    #  values to avoid duplicated function calls
    val = f(a)
    next_val = f(x)

    #loop over the interval of integration being sure not
    #  to overrun the right edge
    while x < b-dx:
        #adding to the running integral 
        tot += 0.5 * (val + next_val)
        #shifting to the next function value
        val = next_val
        #incrementing x
        x += dx
        #computing next function value
        next_val = f(x)

    #returning while multiplying by common dx factor
    #rather than estimating second derivatives to get an error
    # estimate, I use a very bad estimation assuming f'' ~ dx
    return tot*dx, tot*dx**3 / 2



# Function for Simpson's rule for integration
# Inputs: - f: 1d function to integrate
#         - a: lower bound
#         - b: upper bound
#         - n: number of intervals to divide the integration into
# Returns: - the definite integral result
#          - err: an estimate of the error on tot
def simpson(f, a, b, n):
    dx = (b - a)/n
    dx3 = dx**3
    tot, err = 0, 0
    tot += f(a) + f(b)
    err += tot * dx**3
    x = a + dx
    while x < b:
        fx = f(x)
        tot += 4*fx
        #err += fx*dx3
        x += 2*dx
    x = a+2*dx
    while x < b:
        fx = f(x)
        tot += 2*fx
        #err += fx*dx3
        x += 2*dx

    #returning while multiplying by common dx factor
        #rather than estimating third derivatives to get an error
        # estimate, I use a very bad estimation assuming f''' ~ dx
    return tot * dx / 3, tot * dx**4 / 6


In [4]:
f = lambda x: x**(-3/2)
a, b = 1, 5

In [42]:
t1, e1, step = rect(f, a, b, 100)
t2, e2 = rect_vec(f, a, b, 100)
t3, e3 = rect_cen(f, a, b, 100)
tt, et = trap(f, a, b, 100)
q, qe, *_ = quad(f, a, b)
s, se = simpson(f, a, b, 100)
print(t1, e1)
print(t2, e2)
print(t3, e3)
print(tt, et)
print(q, qe)
print(s, se)

1.123980330319958 0.01821114561800018
1.12398033031996 0.018189462681181722
1.1054746386160417 0.0005853422945750131
1.0985265938232416 0.0017576425501171866
1.1055728090000843 5.444307667957722e-13
1.105572994660321 7.075667165826054e-05


In [45]:
blah = np.array([trap(f, a, b, N) for N in np.linspace(5,1000, 40)])

In [61]:
blah = np.hstack((blah, (blah[:,0] - truth).reshape((-1,1))))

In [29]:
%timeit t1, e1, step = rect(f, a, b, 100)
%timeit t2, e2 = rect_vec(f, a, b, 100)
#%timeit tt, et = trap(f, a, b, 100)
#%timeit q, qe, *_ = quad(f, a, b)

8.1 µs ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
9.36 µs ± 52.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [62]:
blah

array([[ 9.85895074e-01,  6.30972848e-01, -1.19677735e-01],
       [ 1.08938915e+00,  1.87213999e-02, -1.61836549e-02],
       [ 1.09957527e+00,  5.60494401e-03, -5.99753786e-03],
       [ 1.09904015e+00,  2.64489547e-03, -6.53266388e-03],
       [ 1.10220983e+00,  1.53886499e-03, -3.36297610e-03],
       [ 1.10143312e+00,  1.00282751e-03, -4.13969231e-03],
       [ 1.10320401e+00,  7.06380768e-04, -2.36879843e-03],
       [ 1.10251686e+00,  5.23369477e-04, -3.05595144e-03],
       [ 1.10372526e+00,  4.03889209e-04, -1.84754521e-03],
       [ 1.10313492e+00,  3.20653246e-04, -2.43789005e-03],
       [ 1.10404609e+00,  2.61055170e-04, -1.52671966e-03],
       [ 1.10353435e+00,  2.16403694e-04, -2.03846021e-03],
       [ 1.10426342e+00,  1.82491366e-04, -1.30938965e-03],
       [ 1.10381371e+00,  1.55817248e-04, -1.75909503e-03],
       [ 1.10442037e+00,  1.34712127e-04, -1.15243941e-03],
       [ 1.10402007e+00,  1.17522923e-04, -1.55273445e-03],
       [ 1.10453903e+00,  1.03506982e-04

In [60]:
asdf

array([-0.11967773, -0.01618365, -0.00599754, -0.00653266, -0.00336298,
       -0.00413969, -0.0023688 , -0.00305595, -0.00184755, -0.00243789,
       -0.00152672, -0.00203846, -0.00130939, -0.0017591 , -0.00115244,
       -0.00155273, -0.00103378, -0.00139407, -0.00094092, -0.00126828,
       -0.00086628, -0.0011661 , -0.00080496, -0.00108146, -0.00075371,
       -0.0010102 , -0.00071022, -0.00094938, -0.00067285, -0.00089686,
       -0.00064041, -0.00085105, -0.00061197, -0.00081075, -0.00058684,
       -0.00077501, -0.00056447, -0.0007431 , -0.00054443, -0.00035602])

In [58]:
len(blah[:,0])

40